In [1]:
import pandas as pd
import numpy as np
import pickle
import warnings

from collections import Counter
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score, confusion_matrix

from IPython.core.interactiveshell import InteractiveShell
warnings.filterwarnings('ignore')
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
cat_feat = pickle.load(open('./data/01_cat_feat.pkl', 'rb'))
num_feat = pickle.load(open('./data/01_num_feat.pkl', 'rb'))
comb_feat = pickle.load(open('./data/01_comb_feat.pkl', 'rb'))
target = pickle.load(open('./data/01_target.pkl', 'rb'))

# Baseline model

## validation

In [3]:
def validation(X_all, y_all, models):
    
    X_train, X_test, y_train, y_test = train_test_split(comb_feat, target, stratify=target, test_size=0.1, random_state=100)
    
    res = []
    
    for model_class in models:
        model = model_class()
        model_name = model.__class__.__name__
        
        cv_score = cross_val_score(model, X_all, y_all, cv=10)
        
        model = model_class()
        _ = model.fit(X_train, y_train)
        pred =model.predict(X_test)
        
        acc = accuracy_score(y_test, pred)
        f1 = f1_score(y_test, pred)
        precision = precision_score(y_test, pred)
        recall = recall_score(y_test, pred)
        
        res.append({
            "model": model_name,
            "cv_score": cv_score.mean(),
            "acc": acc,
            "f1": f1,
            "precision": precision,
            "recall": recall
        })
        
    return res

In [4]:
models = [SVC, XGBClassifier, LogisticRegression, KNeighborsClassifier, DecisionTreeClassifier, RandomForestClassifier, GradientBoostingClassifier]

* combine features

In [5]:
pd.DataFrame(validation(comb_feat, target, models))

,model,cv_score,acc,f1,precision,recall
0,SVC,0.854624,0.903226,0.914286,0.888889,0.941176
1,XGBClassifier,0.788065,0.838710,0.864865,0.800000,0.941176
2,LogisticRegression,0.844624,0.903226,0.914286,0.888889,0.941176
3,KNeighborsClassifier,0.811828,0.806452,0.823529,0.823529,0.823529
4,DecisionTreeClassifier,0.718925,0.838710,0.864865,0.800000,0.941176
5,RandomForestClassifier,0.844839,0.870968,0.888889,0.842105,0.941176
6,GradientBoostingClassifier,0.807849,0.870968,0.888889,0.842105,0.941176


* categorical features

In [6]:
pd.DataFrame(validation(cat_feat, target, models))

,model,cv_score,acc,f1,precision,recall
0,SVC,0.851075,0.903226,0.914286,0.888889,0.941176
1,XGBClassifier,0.831183,0.838710,0.864865,0.800000,0.941176
2,LogisticRegression,0.834409,0.903226,0.914286,0.888889,0.941176
3,KNeighborsClassifier,0.828280,0.806452,0.823529,0.823529,0.823529
4,DecisionTreeClassifier,0.781935,0.806452,0.833333,0.789474,0.882353
5,RandomForestClassifier,0.834839,0.870968,0.888889,0.842105,0.941176
6,GradientBoostingClassifier,0.824624,0.903226,0.914286,0.888889,0.941176


* numeric features

In [7]:
pd.DataFrame(validation(num_feat, target, models))

,model,cv_score,acc,f1,precision,recall
0,SVC,0.735699,0.903226,0.914286,0.888889,0.941176
1,XGBClassifier,0.646237,0.838710,0.864865,0.800000,0.941176
2,LogisticRegression,0.729677,0.903226,0.914286,0.888889,0.941176
3,KNeighborsClassifier,0.669892,0.806452,0.823529,0.823529,0.823529
4,DecisionTreeClassifier,0.643441,0.741935,0.789474,0.714286,0.882353
5,RandomForestClassifier,0.699570,0.838710,0.864865,0.800000,0.941176
6,GradientBoostingClassifier,0.690000,0.903226,0.914286,0.888889,0.941176


# Stacking models

* simple stacking

In [8]:
def train(X, y, models):
    
    res_model = []
    
    for model_class in models:
        model = model_class()
        _ = model.fit(X_train, y_train)
        
        res_model.append(model)
        
    return res_model

In [9]:
X_train, X_test, y_train, y_test = train_test_split(comb_feat, target, stratify=target, test_size=0.1, random_state=100)

In [10]:
models = [SVC, LogisticRegression, GradientBoostingClassifier]

In [11]:
base_models = train(X_train, y_train, models)

In [12]:
def stack_output(X, models):
    res = []
    for model in models:
        pred = model.predict(X).reshape((-1, 1))
        res.append(pred)
    return np.hstack(res)

def stack_output_v2(X, models):
    res = []
    for model in models:
        pred = model.predict(X).reshape((-1, 1))
        res.append(pred)
    return np.hstack(res + [X])

In [13]:
train_out_base = stack_output(X_train, base_models)
train_out_base.shape

(272, 3)

In [14]:
meta_model = SVC()
_ = meta_model.fit(train_out_base, y_train)

In [15]:
test_out_base = stack_output(X_test, base_models)
test_out_base

array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [0, 0, 0],
       [1, 1, 1],
       [0, 0, 0],
       [0, 0, 0],
       [0, 1, 1],
       [0, 0, 0],
       [1, 1, 1],
       [0, 0, 0],
       [0, 0, 0],
       [1, 1, 1],
       [1, 1, 1],
       [0, 0, 0],
       [1, 1, 1],
       [0, 0, 0],
       [1, 1, 1],
       [0, 0, 1],
       [1, 1, 1],
       [0, 0, 0],
       [1, 1, 1],
       [1, 1, 1],
       [0, 0, 0],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 0, 0],
       [1, 1, 1],
       [0, 0, 0]], dtype=int64)

In [16]:
meta_model.score(test_out_base, y_test)

0.8709677419354839

* majority vote

In [17]:
res = []

for row in test_out_base:
    res.append(Counter(row).most_common(1)[0][0])
    
res = np.array(res).reshape((-1, 1))
# res

In [18]:
accuracy_score(y_test, res)
print(classification_report(y_test, res))
confusion_matrix(y_test, res)

0.9032258064516129

              precision    recall  f1-score   support

           0       0.92      0.86      0.89        14
           1       0.89      0.94      0.91        17

    accuracy                           0.90        31
   macro avg       0.91      0.90      0.90        31
weighted avg       0.90      0.90      0.90        31



array([[12,  2],
       [ 1, 16]], dtype=int64)

* 3 meta and majorityy vote

In [19]:
models = [SVC, LogisticRegression, GradientBoostingClassifier]

In [20]:
base_models = train(X_train, y_train, models)

In [21]:
train_out_base1 = stack_output(X_train, base_models)
train_out_base1.shape

train_out_base2 = stack_output_v2(X_train, base_models)
train_out_base2.shape

(272, 3)

(272, 124)

In [22]:
test_out_base1 = stack_output(X_test, base_models)
test_out_base1.shape

test_out_base2 = stack_output_v2(X_test, base_models)
test_out_base2.shape

(31, 3)

(31, 124)

In [23]:
meta_model1 = SVC()
meta_model2 = SVC()
meta_model3 = SVC()

_ = meta_model1.fit(train_out_base1, y_train)
_ = meta_model2.fit(train_out_base2, y_train)
_ = meta_model3.fit(X_train, y_train)

meta_model1.score(test_out_base1, y_test)
meta_model2.score(test_out_base2, y_test)
meta_model3.score(X_test, y_test)

pred1 = meta_model1.predict(test_out_base1).reshape((-1, 1))
pred2 = meta_model2.predict(test_out_base2).reshape((-1, 1))
pred3 = meta_model3.predict(X_test).reshape((-1, 1))

pred = np.hstack([pred1, pred2, pred3])

0.9032258064516129

0.9032258064516129

0.9032258064516129

In [24]:
res = []

for row in pred:
    res.append(Counter(row).most_common(1)[0][0])
    
res = np.array(res).reshape((-1, 1))
# res

In [25]:
accuracy_score(y_test, res)
print(classification_report(y_test, res))
confusion_matrix(y_test, res)

0.9032258064516129

              precision    recall  f1-score   support

           0       0.92      0.86      0.89        14
           1       0.89      0.94      0.91        17

    accuracy                           0.90        31
   macro avg       0.91      0.90      0.90        31
weighted avg       0.90      0.90      0.90        31



array([[12,  2],
       [ 1, 16]], dtype=int64)